In [2]:
import nltk
import pandas as pd

from nltk.sentiment.vader import SentimentIntensityAnalyzer
from toolbox import ToolBox
from textblob import TextBlob

# nltk.download('vader_lexicon')

In [3]:
sid = SentimentIntensityAnalyzer()
sid.polarity_scores('This is not cool')

{'neg': 0.395, 'neu': 0.605, 'pos': 0.0, 'compound': -0.2411}

In [4]:
tb = ToolBox()

#### Example

In [5]:
### DataFrame Pre-Sentiment Analysis ###
reviews.head()

Loading from database


,game,username,date,grade,review,lang,helpful_nb,helpful_nb_total
0,007-legends-pc,Bughyman1000,2013-01-19,30,"Oh, my dear God! What is it with developers th...",en,0,0
1,007-legends-pc,DiabloZiri,2014-06-23,30,Seriously if you want to have a HUGE laugh wit...,en,0,0
2,007-legends-pc,DrugsMeazureTim,2014-12-11,0,godawful port of the xbox 360 version and ps3....,en,0,0
3,007-legends-pc,DustEater,2012-11-03,0,Agree. Worst game ever. Its a full copy of Cal...,en,8,8
4,007-legends-pc,evry1isacritic,2012-11-04,0,DO NOT BUY THIS GAME for the PC (or for any ot...,en,2,2


In [9]:
### DataFrame Post-Sentiment Analysis ###
new_df = vader_sentiment_to_df(reviews[0:5])
new_df.head()

,game,date,grade,review,sentiment
0,007-legends-pc,2013-01-19,30,"Oh, my dear God! What is it with developers th...",-0.9191
1,007-legends-pc,2014-06-23,30,Seriously if you want to have a HUGE laugh wit...,0.9424
2,007-legends-pc,2014-12-11,0,godawful port of the xbox 360 version and ps3....,-0.3869
3,007-legends-pc,2012-11-03,0,Agree. Worst game ever. Its a full copy of Cal...,-0.4854
4,007-legends-pc,2012-11-04,0,DO NOT BUY THIS GAME for the PC (or for any ot...,-0.9612


In [7]:
def vader_sentiment_to_df(dataframe):
    '''
        Gets the sentiment scores of reviews using nltk.sentiment.vader.
        
        Input: dataframe with reviews
        Output: original dataframe + column with sentiment score
    '''
    
    sid = SentimentIntensityAnalyzer()
    compound = []
    
    # iterate over dataframe and calculate sentiment scores per row
    for idx, row in dataframe.iterrows():
        
        scores = list(sid.polarity_scores(row['review']).values())
        
        comp = scores[3]
        
        compound.append(comp)
    
    # turns compound list into a dataframe
    compound_df = pd.DataFrame.from_dict({'compound':compound})
    
    # adds sentiment column to existing dataframe
    new_dataframe = pd.concat([dataframe, compound_df], axis = 1)
    
    # drops unnecessary columsn
    new_dataframe = new_dataframe.drop(columns = ['username', 'lang', 'helpful_nb', 'helpful_nb_total'])
    
    # renames compound column to match database
    new_dataframe.rename(columns={'compound':'sentiment'}, inplace=True)
    new_dataframe.rename(columns={'index':'id'}, inplace=True)
    
    return new_dataframe

In [ ]:
new_df.head()

In [ ]:
# write new_df to pickle file
new_df.to_pickle('user_review_clean.pkl')

In [ ]:
# load pickle file
new_df = pd.read_pickle('user_review_clean.pkl')

#### Trying to load dataframe into the database.....fail

In [ ]:
from models import *

In [ ]:
new_df['date'] = new_df['date'].astype(str)

In [ ]:
data = new_df.to_dict(orient='records')

In [ ]:
# Insert rows 100 at a time.
with database.atomic():
    for idx in range(0, len(data), 1000):
        UserReviewsClean.insert_many(data[idx:idx+1000]).execute()

### Experimentation with Flair

In [ ]:
import flair
from flair.data import Sentence

flair_sentiment = flair.models.TextClassifier.load('en-sentiment')

In [ ]:
review = reviews[reviews['grade'] > 90].iloc[12]['review']

print(review)

s = flair.data.Sentence(review)
flair_sentiment.predict(s)

In [ ]:
tagger = flair.models.SequenceTagger.load('nl-ner')

In [ ]:
sentence = Sentence('Eetzaal van de aan het klooster verbonden kweekschool. De meisjes die daar werden opgeleid tot onderwijzeres verbleven intern in een internaat. De enige versieringen in deze zaal bestaan uit religieuze voorstellingen, waaronder een Mariabeeld en een H. Hartbeeld.')

# predict NER tags
tagger.predict(sentence)

# print sentence with predicted tags
# print(sentence.to_tagged_string())

In [ ]:
for entity in sentence.get_spans('ner'):
    print(entity)

In [ ]:
raw = "Sint-Janssingel gezien in zuidelijke richting ter hoogte van de Stationsbrug, later Wilhelminabrug. Midden op de achtergrond de molen bij het Wilhelminapark. Rechts de nieuwe bebouwing aan de Kloostersingel, hoek Luybenstraat. De rest van de nieuwe wijk het Zand is nog onbebouwd. Achter de bebouwing is de kerk van de paters Kapucijnen te zien. Links de Sint-Janssingel met daarachter het kloostercomplex van de Marienburg. Fotograaf is Herman de Ruijter HdR"

sentences = [Sentence(sentence) for sentence in raw.split('.')]

for sentence in sentences:
    print(sentence)
    tagger.predict(sentence)
    
    for entity in sentence.get_spans('ner'):
        print(entity)
        
    print('--------------------------------')